# Bushfire Detection Model Development
Using the Edge Impulse Python SDK with TensorFlow and Keras to train model to detect bushfires vs. no bushfire within a given image.

#Install Dependencies

In [1]:
!python -m pip install tensorflow==2.12.0 edgeimpulse

# Import Packages

In [2]:
from tensorflow import keras
import edgeimpulse as ei
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import os

# Setup

In [29]:
# Settings
ei.API_KEY = "ei_c90f6349e4f2624274eff30ee840a3e6412c3135d1651ee15515404d290ce842"
labels = ["fire", "nofire"]
num_classes = len(labels)
deploy_filename = "bushfire_detection_model_cpp.zip"

In [30]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Prepare Dataset
Organise files between 'Training and Validation' and 'Testing' folders containing 'fire' and 'nofire' images.


In [31]:
# Define paths for your dataset
train_val_directory = 'drive/MyDrive/forest_fire/Training and Validation'
test_directory = 'drive/MyDrive/forest_fire/Testing'

# Create an ImageDataGenerator for training and validation
train_val_datagen = ImageDataGenerator(
    rescale=1.0 / 255,        # Normalize pixel values
    validation_split=0.2      # Use 20% of the training data for validation
)

# Create generators for training and validation data
train_generator = train_val_datagen.flow_from_directory(
    train_val_directory,
    target_size=(128, 128),   # Resize images to 128x128
    batch_size=32,             # Number of images to return in each batch
    class_mode='binary',       # Binary classification (fire or no fire)
    subset='training',          # Set as training data
    classes=['fire', 'nofire'] # Explicitly specify the class names
)

validation_generator = train_val_datagen.flow_from_directory(
    train_val_directory,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',       # Binary classification
    subset='validation',        # Set as validation data
    classes=['fire', 'nofire'] # Explicitly specify the class names
)

Found 1467 images belonging to 2 classes.
Found 365 images belonging to 2 classes.


Setup of 'Testing' directory

In [32]:
# Create an ImageDataGenerator for testing (no split needed)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Create a generator for the test data
test_generator = test_datagen.flow_from_directory(
    test_directory,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',       # Binary classification
    shuffle=False,             # Don't shuffle test data
    classes=['fire', 'nofire'] # Explicitly specify the class names
)

Found 68 images belonging to 2 classes.


#Building and compiling model

In [33]:
# Build your model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Testing model setup

In [34]:
print(train_generator.class_indices)

{'fire': 0, 'nofire': 1}


# Training Model

In [35]:
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10
)

Epoch 1/10
45/45 [==============================] - 35s 621ms/step - loss: 0.5880 - accuracy: 0.8300 - val_loss: 0.5059 - val_accuracy: 0.8068
Epoch 2/10
45/45 [==============================] - 29s 632ms/step - loss: 0.3702 - accuracy: 0.8787 - val_loss: 0.3932 - val_accuracy: 0.8267
Epoch 3/10
45/45 [==============================] - 28s 620ms/step - loss: 0.2848 - accuracy: 0.8906 - val_loss: 0.3553 - val_accuracy: 0.8580
Epoch 4/10
45/45 [==============================] - 29s 630ms/step - loss: 0.2467 - accuracy: 0.9038 - val_loss: 0.3378 - val_accuracy: 0.8722
Epoch 5/10
45/45 [==============================] - 28s 631ms/step - loss: 0.2233 - accuracy: 0.9136 - val_loss: 0.3134 - val_accuracy: 0.8864
Epoch 6/10
45/45 [==============================] - 28s 627ms/step - loss: 0.1980 - accuracy: 0.9254 - val_loss: 0.3041 - val_accuracy: 0.8864
Epoch 7/10
45/45 [==============================] - 28s 620ms/step - loss: 0.1859 - accuracy: 0.9289 - val_loss: 0.2915 - val_accuracy: 0.8892

#Test Accuracy and Evaluation

In [36]:
# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f'Test Accuracy: {test_accuracy:.2f}')

2/2 [==============================] - 1s 193ms/step - loss: 0.3167 - accuracy: 0.8750
Test Accuracy: 0.88


# Model to System Compatibility

In [24]:
# List the available profile target devices
# ei.model.list_profile_devices()

In [12]:
# Estimate the RAM, ROM, and inference time for our model on the target hardware family
# try:
#     profile = ei.model.profile(model=model,
#                                device='openmv-h7p')
#     print(profile.summary())
# except Exception as e:
#     print(f"Could not profile: {e}")

# Deploying and Downloading Model

In [37]:
# Define the output and input types for your model
model_output_type = ei.model.output_type.Classification(labels=labels)

# Define the model input type for images
model_input_type = ei.model.input_type.ImageInput()

# Define the filename for the deployment package
deploy_filename = "saved_model_zip.zip"

# Save your model as a TensorFlow SavedModel first
model.save("saved_model")  # Save model in the correct format

# Try to deploy the model directly as a C++ library in a .zip file
deploy_bytes = None
try:
    # Specify the path to the saved model
    deploy_bytes = ei.model.deploy(
        model="saved_model",  # Path to the saved model
        model_output_type=model_output_type,
        model_input_type=model_input_type,
        deploy_target='zip'  # Export as a .zip file with C++ library
    )
    print("Model deployment successful.")
except Exception as e:
    print(f"Could not deploy: {e}")

# Write the deployed model bytes to a file
if deploy_bytes:
    with open(deploy_filename, 'wb') as f:
        f.write(deploy_bytes.getvalue())
    print(f"Model exported as {deploy_filename}")
else:
    print("Deployment failed; no deployable bytes were generated.")

Model deployment successful.
Model exported as saved_model_zip.zip


In [38]:
from google.colab import files
import shutil

shutil.make_archive("saved_model", 'zip', "saved_model")
files.download("saved_model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Model Specifications
Code to check if model has been trained correctly

In [39]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 63, 63, 32)       0         
 2D)                                                             
                                                                 
 global_average_pooling2d_2   (None, 32)               0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_4 (Dense)             (None, 64)                2112      
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                                 
Total params: 3,073
Trainable params: 3,073
Non-traina